In [ ]:
!pip install -r requirements --quiet

import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Data Setup

The goal here is to take some bitcoin data, and predict the next few points of data

The input were going to use is
- H consecutive points of bitcoin pricing data, 
- with C columns (ie: open, close, high, low, volume)
- and I indicators (columns calculated from others, ie: Bollinger Bands, MA, EMA, etc)

with total shape (H,C+I), or (B,H,C+I) if were doing batches of suze B

the output is then going to be a set of L output points
- L (lookahead) meaning we want to predict L points into the future,
- C (columns)  (ie: open, close, high, low, volume)
- I: indicators (optional) (it is useful to predict these if we want to propogate the model off its own output)

so total output of (L,C+I?) or (B,L,C+I?) if batches

In [ ]:
from data.CsvDataConfig import CalculatedColumnConfig, CsvDataConfig
from data.wrappers.CsvDataWrapper import CsvDataWrapper
from legitindicators import bollinger_bands_pb, rma,voss, linreg_curve

data_columns = ["Open","Close","Number of Trades"]

indicators = [
    CalculatedColumnConfig("Voss",voss,["Close"],dict(period=20, predict=3, bandwidth=.25)),
    CalculatedColumnConfig("Bollinger",bollinger_bands_pb,["Close"],dict(length=25,stdd=3)),
    CalculatedColumnConfig("Linear Regression Curve",linreg_curve,["Close"],dict(length=25)),
]

data_config = CsvDataConfig(data_columns,indicators,horizon=30,lookahead=2,batch_size=5,num_batches=20, predict_calculated=True)
train_data = CsvDataWrapper.load_from_file('test_input/bitcoin_jan_2022_1h.csv',index_name='Open Time',indicators=indicators)
test_data = CsvDataWrapper.load_from_file('test_input/bitcoin_feb_2022_1h.csv',index_name='Open Time',indicators=indicators)

This provides data with shapes
input shape: (5,30,6)
output shape:(5,2,6) 

### Now to build the Model

In [ ]:
from models.CsvModel import CsvModel
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MSE

a = 0.08
layers = [Dense(200),LeakyReLU(a),Dense(200),LeakyReLU(a),Dense(200),LeakyReLU(a),Dense(100),LeakyReLU(a),Dense(50),LeakyReLU(a)]
optimizer = Adam(learning_rate=5e-4)
loss = MSE
csv_model = CsvModel(data_config.input_shape, data_config.output_shape, layers, optimizer, loss)
csv_model.build()
csv_model.set_dataconfig(data_config)
csv_model.set_test_datawrapper(test_data)
csv_model.set_train_datawrapper(train_data)

### And now we train!

In [ ]:
epochs=2000
trains_per_test=20

from plotting.TrainPlotter import TrainPlotter
from plotting import TestPlotter

train_columns = ["Train " + x + " Loss" for x in data_columns]
loss_plot = TrainPlotter(moving_average_size=trains_per_test*4,labels=[*train_columns,"Test Loss"])

test_loss = 0
for i in range(epochs):
    loss_plot.start_epoch()
    train_loss = csv_model.train()

    if i % trains_per_test == 0 and i != 0:
        test_result = csv_model.test()
        test_loss = test_result.loss
        TestPlotter.plot_test_result('test_output/bitcoin_test.png', test_result)
        TestPlotter.propgate_and_plot('test_output/propogate_test.png', csv_model, train_data, data_config, 24)

    loss_plot.batch_update([*train_loss[data_columns].sum(),test_loss])
    loss_plot.log_epoch()